In [1]:
import random
import sqlite3

def extract_from_db(cursor, id: int, include_parent=False):
    cursor.execute("SELECT * FROM laws WHERE id = ?", (id,))
    rows = cursor.fetchall()

    columns = [col[0] for col in cursor.description]
    results = [dict(zip(columns, row)) for row in rows]

    if include_parent:
        all_nodes = []
        for r in results:
            current = r
            while current['parent_id'] is not None:
                cursor.execute("SELECT * FROM laws WHERE id = ?", (current['parent_id'],))
                parent = cursor.fetchone()
                if parent is None:
                    break
                parent_dict = dict(zip(columns, parent))
                all_nodes.append(parent_dict)
                current = parent_dict
        results.extend(all_nodes)

    return results[::-1]

def extract_random_from_db(cursor, include_parent=True):
    cursor.execute("SELECT * FROM laws WHERE title LIKE '%Điểm%'")
    rows = cursor.fetchall()
    if not rows:
        return []

    columns = [col[0] for col in cursor.description]
    row = random.choice(rows)
    result = dict(zip(columns, row))

    results = [result]

    if include_parent:
        current = result
        while current['parent_id'] is not None:
            cursor.execute("SELECT * FROM laws WHERE id = ?", (current['parent_id'],))
            parent = cursor.fetchone()
            if parent is None:
                break
            parent_dict = dict(zip(columns, parent))
            results.append(parent_dict)
            current = parent_dict

    return results[::-1]

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

def get_gpt_response(user_prompt, system_prompt, api_key, model="gpt-4o"):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

In [3]:
def clean_text(text: str) -> str:
    """Remove newlines, tabs, extra spaces, punctuation and lowercase."""
    text = re.sub(r"[\r\n\t]+", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[!?]+", "", text)
    return text.strip().lower()

In [4]:
conn = sqlite3.connect(r"E:\Github\LawAssistant\triplet_extraction\law.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Các bảng trong law.db:", tables)

Các bảng trong law.db: [('laws',), ('law_refs',)]


In [5]:
rows = extract_random_from_db(cursor, True)
law = ""
title = ""
for r in rows:
    title += r['title'] + " "
    if not (r['title'].strip().startswith("Chương") or  r['title'].strip().startswith("Điều")):
        law += r['content'] + "\n"

so_hieu = r['so_hieu']
id = r['id']
print(id)
print(so_hieu + " " + title)
print(clean_text(law))

80a675aa13c6bad025f55461c749be4d0f554787e7f2254b6ad85561a3029755
92/2015/QH13 Chương XXXIX Điều 482 Khoản 2 Điểm a 
những bản án, quyết định sau đây của tòa án cấp sơ thẩm được thi hành ngay mặc dù có thể bị kháng cáo, khiếu nại, kháng nghị, kiến nghị: bản án, quyết định về cấp dưỡng, trả công lao động, nhận người lao động trở lại làm việc, trả lương, trợ cấp thôi việc, trợ cấp mất sức lao động, trợ cấp mất việc làm, bảo hiểm xã hội, bảo hiểm thất nghiệp, bảo hiểm y tế hoặc bồi thường thiệt hại về tính mạng, sức khoẻ, tổn thất tinh thần của công dân; quyết định về tính hợp pháp của cuộc đình công;


In [6]:
rows = extract_from_db(cursor, "b7fee66c5d809a5fd7023b6810338639e4b19563187a93e21e17d4e21e92da77", True)
law = ""
title = ""
for r in rows:
    title += r['title'] + " "
    if not (r['title'].strip().startswith("Chương") or r['title'].strip().startswith("Điều")):
        law += r['content'] + "\n"

so_hieu = r['so_hieu']
id = r['id']
print(id)
print(so_hieu + " " + title)
print(clean_text(law))

b7fee66c5d809a5fd7023b6810338639e4b19563187a93e21e17d4e21e92da77
36/2024/QH15 Chương IV Điều 57 Khoản 1 Điểm p 
giấy phép lái xe bao gồm các hạng sau đây: hạng de cấp cho người lái các loại xe ô tô quy định cho giấy phép lái xe hạng d kéo rơ moóc có khối lượng toàn bộ theo thiết kế trên 750 kg; xe ô tô chở khách nối toa.


In [12]:
system_prompt_rewrite = """
Bạn là trợ lý AI Tiếng Việt chuyên nghiệp và trung thực.
Bạn là chuyên gia pháp luật Việt Nam, am hiểu các bộ luật, nghị định, và văn bản pháp luật.
Bạn là chuyên gia ngôn ngữ Việt Nam, biết viết câu chuẩn cấu trúc, chính xác, trang trọng, và đúng ngôn ngữ pháp lý.
Luôn trả lời chính xác, hữu ích, ngắn gọn và an toàn.
Nếu thông tin không hợp lý hoặc thiếu, hãy yêu cầu thêm thông tin thay vì đoán mò.
Không thay đổi ý nghĩa khi viết lại câu.
Luôn dùng ngôn ngữ chính xác như trong văn bản pháp luật, tránh ngôn ngữ thông thường hay không trang trọng.

Định nghĩa 'câu đơn': Một câu đơn là câu có một chủ ngữ (hoặc cụm chủ ngữ) và một vị ngữ (hoặc cụm vị ngữ), biểu đạt một ý trọn vẹn; câu có thể chứa thành tố phụ (tính từ, trạng từ, bổ ngữ) nhưng không được ghép bằng liên từ hoặc dấu câu như dấu ",", ";" để tạo hai hoặc nhiều mệnh đề độc lập.

Quy tắc bắt buộc:
1. Khi viết lại, **chỉ** trả về các câu đơn theo đúng định nghĩa trên; mỗi câu một dòng nếu có nhiều câu.
2. **Được phép tái sử dụng** các thành phần câu (chủ ngữ, cụm danh từ, đại từ, cụm tính từ, v.v.) từ vế trước hoặc từ phần khác của câu gốc để hoàn chỉnh vế thiếu, **nhằm bảo toàn ý nghĩa** sau khi tách.
3. Khi tái sử dụng, **ưu tiên giữ nguyên** từ ngữ gốc; chỉ thực hiện điều chỉnh nhỏ cần thiết để tạo câu đơn ngữ pháp đúng, **không** thêm thông tin, suy đoán hay nội dung mới.
4. Tuyệt đối không kèm chú giải, giải thích, danh sách hay bất kỳ nội dung nào khác ngoài các câu viết lại.
5. Nếu câu gốc mơ hồ hoặc thiếu thông tin đến mức không thể tạo câu đơn hoàn chỉnh mà vẫn giữ nguyên ý, hãy yêu cầu thêm thông tin ngắn gọn.
Danh mục liên từ cần loại trừ khi viết câu đơn: và, hoặc, hoặc là, hay, hay là, nhưng, song, tuy nhiên, mà, còn, rồi.
Giữ nguyên thứ tự trước sau của các từ sau khi viết lại câu.
Sau khi viết lại câu không được thiếu từ danh từ nào trong câu gốc và phải độc lập không phụ thuộc vào câu trước đó.
"""


user_prompt_rewrite = f"""
Ngữ cảnh: Bộ luật số {so_hieu} trong luật Việt Nam
Nhiệm vụ: Viết lại câu sau để hoàn chỉnh cấu trúc với đầy đủ chủ ngữ và vị ngữ, giữ nguyên ý nghĩa. Mỗi câu xuất ra phải là một câu đơn đầy đủ (một dòng một câu nếu có nhiều câu).
Câu cần viết lại: "{clean_text(law)}"
"""


In [13]:
print(user_prompt_rewrite)


Ngữ cảnh: Bộ luật số 36/2024/QH15 trong luật Việt Nam
Nhiệm vụ: Viết lại câu sau để hoàn chỉnh cấu trúc với đầy đủ chủ ngữ và vị ngữ, giữ nguyên ý nghĩa. Mỗi câu xuất ra phải là một câu đơn đầy đủ (một dòng một câu nếu có nhiều câu).
Câu cần viết lại: "giấy phép lái xe bao gồm các hạng sau đây: hạng de cấp cho người lái các loại xe ô tô quy định cho giấy phép lái xe hạng d kéo rơ moóc có khối lượng toàn bộ theo thiết kế trên 750 kg; xe ô tô chở khách nối toa."



In [53]:
rewrite_sentence = get_gpt_response(user_prompt_rewrite, system_prompt_rewrite, api_key, "gpt-4.1-mini")

NameError: name 'user_prompt_rewrite' is not defined

In [7]:
rewrite_sentence = """
Giấy phép lái xe bao gồm các hạng sau đây.
Giấy phép lái xe hạng D được cấp cho người lái các loại xe ô tô quy định cho giấy phép lái xe hạng D.
Giấy phép lái xe hạng D được cấp cho người lái rơ moóc có khối lượng toàn bộ theo thiết kế trên 750 kg.
Giấy phép lái xe hạng D được cấp cho người lái xe ô tô chở khách nối toa.
"""

In [15]:
vncorenlp_pos_map = {
    "N":   "Noun (Danh từ)",
    "Np":  "Proper noun (Danh từ riêng)",
    "Nc":  "Classifier noun (Danh từ giống loại)",
    "Nu":  "Unit noun (Danh từ đơn vị)",
    "V":   "Verb (Động từ)",
    "Vb":  "Verb (base) (Động từ gốc)",
    "A":   "Adjective (Tính từ)",
    "Ai":  "Adjective (predicative) (Tính từ vị ngữ)",
    "P":   "Pronoun (Đại từ)",
    "R":   "Adverb (Trạng từ)",
    "M":   "Numeral / number (Số từ)",
    "E":   "Preposition / particle (Giới từ / trợ từ)",
    "C":   "Coordinating conjunction (Liên từ phối hợp)",
    "CC":  "Subordinating conjunction / complementizer (Liên từ phụ thuộc)",
    "L":   "Determiner / article (Từ hạn định)",
    "D":   "Adverbial marker / degree marker (Từ chỉ mức độ)",
    "X":   "Other (Khác)",
    "CH":  "Punctuation (Dấu câu)"
}

def process_sentence(text: str, rdrsegmenter, verbose: bool = True) -> dict:
    results = {
        "original": text,
        "cleaned": "",
        "segmented": [],
        "pos_annotation": [],
        "concepts": []
    }

    if verbose:
        print("1. Original text:\n", text, "\n")

    # 2. Clean
    text = clean_text(text)
    results["cleaned"] = text
    if verbose:
        print("2. Cleaned text:\n", text, "\n")

    # 3. Segmentation
    segmented = rdrsegmenter.word_segment(text)
    results["segmented"] = segmented
    if verbose:
        print("3. Segmented text (tokens):\n", segmented, "\n")

    # 4. POS tagging
    output = rdrsegmenter.annotate_text(text)
    sents = output.values() if isinstance(output, dict) else output

    pos_annot = []
    tokens = []

    if verbose:
        print("4. POS & Head & DepRel annotation:")
        print(f"{'Idx':<5} {'Token':<15} {'Head':<5} {'DepRel':<15} {'POS':<25}")
        print("-" * 75)

    for sent in sents:
        if not isinstance(sent, list):
            continue
        for token in sent:
            if not isinstance(token, dict):
                continue
            word = token.get("wordForm", "")
            pos = token.get("posTag", "")
            head = token.get("head", "")
            dep = token.get("depLabel", "")
            pos_full = vncorenlp_pos_map.get(pos, pos)

            pos_data = {
                "index": token.get("index", ""),
                "token": word,
                "pos": pos_full,
                "head": head,
                "dep": dep
            }
            pos_annot.append(pos_data)

            if verbose:
                print(f"{pos_data['index']:<5} {pos_data['token']:<15} {pos_data['head']:<5} {pos_data['dep']:<15} {pos_data['pos']:<25}")

            # Only append N, V, A, E for triplet extraction
            if pos.startswith(("N", "V", "A", "E", "M")):
                tokens.append((word.replace("_", " "), pos))

    results["pos_annotation"] = pos_annot

    # 5. Triplet extraction
    concepts = []
    concept1_tokens = []
    concept2_tokens = []
    relation_tokens = []

    for token, pos in tokens:
        if pos.startswith("V"):
            if concept2_tokens:
                triplet = (
                    " ".join(concept1_tokens),
                    " ".join(relation_tokens),
                    " ".join(concept2_tokens)
                )
                if triplet not in concepts:  # chỉ thêm nếu chưa tồn tại
                    concepts.append(triplet)

                concept1_tokens = concept2_tokens
                concept2_tokens = []
                relation_tokens = []

            relation_tokens.append(token)
        else:
            if relation_tokens:
                concept2_tokens.append(token)
            else:
                concept1_tokens.append(token)

    # Append last triplet if complete
    if concept1_tokens and relation_tokens and concept2_tokens:
        concepts.append((
            " ".join(concept1_tokens),
            " ".join(relation_tokens),
            " ".join(concept2_tokens)
        ))

    results["concepts"] = concepts

    if verbose:
        print("\n5. Extracted triplets:")
        for c in concepts:
            print(c)

    return results

In [41]:
import re

# giữ map POS của bạn
vncorenlp_pos_map = {
    "N":   "Noun (Danh từ)",
    "Np":  "Proper noun (Danh từ riêng)",
    "Nc":  "Classifier noun (Danh từ giống loại)",
    "Nu":  "Unit noun (Danh từ đơn vị)",
    "V":   "Verb (Động từ)",
    "Vb":  "Verb (base) (Động từ gốc)",
    "A":   "Adjective (Tính từ)",
    "Ai":  "Adjective (predicative) (Tính từ vị ngữ)",
    "P":   "Pronoun (Đại từ)",
    "R":   "Adverb (Trạng từ)",
    "M":   "Numeral / number (Số từ)",
    "E":   "Preposition / particle (Giới từ / trợ từ)",
    "C":   "Coordinating conjunction (Liên từ phối hợp)",
    "CC":  "Subordinating conjunction / complementizer (Liên từ phụ thuộc)",
    "L":   "Determiner / article (Từ hạn định)",
    "D":   "Adverbial marker / degree marker (Từ chỉ mức độ)",
    "X":   "Other (Khác)",
    "CH":  "Punctuation (Dấu câu)"
}

# ----- Các cấu hình / stoplists / helper sets -----
AUX_WORDS = {"được", "bị", "đang", "đã", "sẽ"}        # trợ động từ hay gặp để gộp với động từ chính
NO_OBJECT_TOKENS = {"sau", "như", "như sau", "ví dụ", "v.v", "v.v."}
DETERMINERS = {"các", "những", "một", "mỗi", "toàn_bộ", "tất_cả", "cả"}
# Những danh từ như "trách nhiệm" mà thường theo sau bởi chuỗi động từ cần kéo vào object
N_HEADS_WITH_VERB_MOD = {"trách nhiệm", "nhiệm vụ", "quyền", "bổn phận", "nghĩa vụ"}

# Các helper
def is_np_pos(pos):
    return isinstance(pos, str) and pos.startswith("N")

def is_verb_pos(pos):
    return isinstance(pos, str) and pos.startswith("V")

def is_prep_pos(pos):
    return isinstance(pos, str) and pos.startswith("E")

def normalize_token(tok: str) -> str:
    return tok.replace("_", " ").strip()

# Gộp trợ động từ + động từ chính nếu có
def merge_aux_verbs(tokens_pos):
    """
    tokens_pos: list of (token_str, pos_tag_short)
    Trả về list đã gộp, ví dụ ('được', 'V') + ('quy định','V') -> ('được quy định', 'V')
    """
    merged = []
    i = 0
    while i < len(tokens_pos):
        tok, pos = tokens_pos[i]
        # nếu token là trợ động từ (được, bị, ...) và tiếp theo là verb => gộp
        if tok in AUX_WORDS and i + 1 < len(tokens_pos):
            nxt_tok, nxt_pos = tokens_pos[i+1]
            if is_verb_pos(nxt_pos):
                merged.append((f"{tok} {nxt_tok}", nxt_pos))
                i += 2
                continue
        # ngược lại, giữ nguyên
        merged.append((tok, pos))
        i += 1
    return merged

# Mở rộng NP (backward/forward). Nếu head là 1 trong N_HEADS_WITH_VERB_MOD, kéo theo verb modifiers
def expand_np_with_post_verbs(tokens_pos, start_idx, direction="forward"):
    """
    start_idx: index to begin expansion (for forward: start_idx is index of the first NP token; for backward: index of last NP token)
    direction: "forward" or "backward"
    """
    np_tokens = []
    i = start_idx
    step = 1 if direction == "forward" else -1
    # collect N / E tokens (simple heuristic)
    while 0 <= i < len(tokens_pos):
        t, p = tokens_pos[i]
        if is_np_pos(p) or is_prep_pos(p) or (p and p.startswith("L")):  # include determiners marked L if present
            if direction == "backward":
                np_tokens.insert(0, t)
            else:
                np_tokens.append(t)
            i += step
        else:
            break

    # If forward expansion and last token is a head that should pull verb modifiers, then pull following verbs
    if direction == "forward" and np_tokens:
        last = normalize_token(np_tokens[-1])
        if last in N_HEADS_WITH_VERB_MOD:
            j = start_idx + len(np_tokens)
            # append consecutive verbs immediately after
            while j < len(tokens_pos):
                t_j, p_j = tokens_pos[j]
                if is_verb_pos(p_j):
                    np_tokens.append(t_j)
                    j += 1
                else:
                    break

    return np_tokens

# Main extractor using POS heuristics + NP expansion + aux merge
def extract_triplets_from_pos_annot(pos_annot):
    """
    pos_annot: list of dicts with at least:
        - "index"
        - "token" (token string, underscores possible)
        - "pos" or "posTag" (short tag like 'N', 'Vb', etc.)
    Returns list of (subject, relation, object)
    """
    # Normalize tokens_pos to list of (token_str, pos_short)
    tokens_pos = []
    for d in pos_annot:
        t = normalize_token(d.get("token", ""))
        # try to obtain short pos tag
        pos_short = d.get("posTag") or d.get("pos") or ""
        if isinstance(pos_short, str) and " " in pos_short:
            pos_short = pos_short.split()[0]
        tokens_pos.append((t, pos_short))

    # Merge auxiliaries
    tokens_pos = merge_aux_verbs(tokens_pos)

    triplets = []
    i = 0
    while i < len(tokens_pos):
        tok, pos = tokens_pos[i]

        # If current is a verb -> candidate relation
        if is_verb_pos(pos):
            relation = tok

            # SUBJECT: look backward for nearest NP (take last NP before verb), include determiner if adjacent
            subj = []
            j = i - 1
            while j >= 0:
                t_j, p_j = tokens_pos[j]
                if is_np_pos(p_j):
                    # expand NP backward (find extent)
                    # find the starting index of this NP (scan backward until not N/E)
                    start_idx = j
                    while start_idx - 1 >= 0 and (is_np_pos(tokens_pos[start_idx - 1][1]) or is_prep_pos(tokens_pos[start_idx - 1][1])):
                        start_idx -= 1
                    subj = expand_np_with_post_verbs(tokens_pos, start_idx, direction="forward")
                    # include determiner immediately before NP if present
                    if start_idx - 1 >= 0:
                        prev_tok, prev_pos = tokens_pos[start_idx - 1]
                        if prev_tok in DETERMINERS:
                            subj.insert(0, prev_tok)
                    break
                elif t_j in DETERMINERS:
                    # keep searching — maybe determiner then noun before that
                    j -= 1
                    continue
                else:
                    j -= 1

            # OBJECT: look forward for nearest NP
            obj = []
            k = i + 1
            while k < len(tokens_pos):
                t_k, p_k = tokens_pos[k]
                if is_np_pos(p_k):
                    # expand NP forward from k
                    obj = expand_np_with_post_verbs(tokens_pos, k, direction="forward")
                    break
                # special-case: relation == "có" -> expect "có <N> <V*>" pattern; treat next N as object
                if relation == "có" and is_np_pos(p_k):
                    obj = expand_np_with_post_verbs(tokens_pos, k, direction="forward")
                    break
                # if sees punctuation or full stop before NP, stop
                if p_k and p_k.startswith("CH"):  # punctuation token class in your map
                    break
                # if next is another verb and no NP later, likely no object for this relation
                if is_verb_pos(p_k):
                    # allow sequences like "phối_hợp giải_quyết" only if they were attached to an earlier head (handled in NP expansion)
                    break
                k += 1

            subj_str = " ".join(subj).strip()
            obj_str = " ".join(obj).strip()

            # Filter meaningless objects
            if obj_str and obj_str not in NO_OBJECT_TOKENS:
                if subj_str:
                    trip = (subj_str, relation, obj_str)
                    if trip not in triplets:
                        triplets.append(trip)
            else:
                # Optionally store incomplete triplets (subject, relation, None) - commented out by default
                # if subj_str:
                #     triplets.append((subj_str, relation, None))
                pass

        i += 1

    return triplets

# ----------------- Phiên bản process_sentence đã cập nhật -----------------
def process_sentence(text: str, rdrsegmenter, verbose: bool = True) -> dict:
    results = {
        "original": text,
        "cleaned": "",
        "segmented": [],
        "pos_annotation": [],
        "concepts": []
    }

    if verbose:
        print("1. Original text:\n", text, "\n")

    # 2. Clean (giữ nguyên clean_text bạn đang dùng)
    # giả định bạn có hàm clean_text — nếu không có hãy thay bằng: text = text.strip().lower()
    try:
        text = clean_text(text)
    except NameError:
        # fallback nhẹ nếu không có clean_text
        text = text.strip().lower()
    results["cleaned"] = text
    if verbose:
        print("2. Cleaned text:\n", text, "\n")

    # 3. Segmentation
    try:
        segmented = rdrsegmenter.word_segment(text)
    except Exception:
        # fallback: treat whole text as one segment
        segmented = [text]
    results["segmented"] = segmented
    if verbose:
        print("3. Segmented text (tokens):\n", segmented, "\n")

    # 4. POS tagging / annotation
    output = rdrsegmenter.annotate_text(text)
    # annotate_text có thể trả dict of sents hoặc list
    sents = output.values() if isinstance(output, dict) else output

    pos_annot = []
    if verbose:
        print("4. POS annotation:")
        print(f"{'Idx':<5} {'Token':<20} {'POS':<30}")
        print("-" * 60)

    for sent in sents:
        if not isinstance(sent, list):
            continue
        for token in sent:
            if not isinstance(token, dict):
                continue
            word = token.get("wordForm", token.get("token", ""))
            pos_short = token.get("posTag", "") or token.get("pos", "")
            pos_full = vncorenlp_pos_map.get(pos_short, pos_short)
            idx = token.get("index", "")

            pos_data = {
                "index": idx,
                "token": word,
                "pos": pos_full,       # full human-readable
                "posTag": pos_short    # short tag for heuristics
            }
            pos_annot.append(pos_data)

            if verbose:
                print(f"{str(idx):<5} {normalize_token(word):<20} {pos_full:<30}")

    results["pos_annotation"] = pos_annot

    # 5. Triplet extraction (dùng hàm mới)
    concepts = extract_triplets_from_pos_annot(pos_annot)
    results["concepts"] = concepts

    if verbose:
        print("\n5. Extracted triplets:")
        for c in concepts:
            print(c)

    return results


In [6]:
def insert_triplet(tx, concept1, relation, concept2, c1_metadata=None, c2_metadata=None, rel_metadata=None):
    query = """
    MERGE (c1:Concept {name: $concept1})
    ON CREATE SET c1 = $c1_metadata
    ON MATCH SET
        c1.document_number = apoc.coll.toSet(coalesce(c1.document_number, []) + [$c1_metadata.document_number]),
        c1.title           = apoc.coll.toSet(coalesce(c1.title, []) + [$c1_metadata.title]),
        c1.document_id     = apoc.coll.toSet(coalesce(c1.document_id, []) + [$c1_metadata.document_id])

    MERGE (c2:Concept {name: $concept2})
    ON CREATE SET c2 = $c2_metadata
    ON MATCH SET
        c2.document_number = apoc.coll.toSet(coalesce(c2.document_number, []) + [$c2_metadata.document_number]),
        c2.title           = apoc.coll.toSet(coalesce(c2.title, []) + [$c2_metadata.title]),
        c2.document_id     = apoc.coll.toSet(coalesce(c2.document_id, []) + [$c2_metadata.document_id])

    MERGE (c1)-[r:RELATION {name: $relation}]->(c2)
    ON CREATE SET r = $rel_metadata
    ON MATCH SET  r += $rel_metadata

    RETURN c1, r, c2
    """
    tx.run(
        query,
        concept1=concept1,
        concept2=concept2,
        relation=relation,
        c1_metadata=c1_metadata or {},
        c2_metadata=c2_metadata or {},
        rel_metadata=rel_metadata or {}
    )

def delete_all(tx):
    # Delete all nodes and relationships
    tx.run("MATCH (n) DETACH DELETE n")

In [7]:
def extract_triplet_and_store(input_text: str, rdrsegmenter, driver, db_name, document_id, document_number):
    with driver.session(database=db_name) as session:
        split_text = input_text.split("\n")
        for t in split_text:
            res = process_sentence(t, rdrsegmenter, verbose=False)
            for concept1, relation, concept2 in res["concepts"]:
                session.execute_write(
                    insert_triplet,
                    concept1,
                    relation,
                    concept2,
                    c1_metadata={"document_number": document_number, "document_id": document_id},
                    c2_metadata={"document_number": document_number, "document_id": document_id},
                    rel_metadata={"document_number": document_number, "document_id": document_id},
                )

In [9]:
def extract_triplet(input_text: str, rdrsegmenter, verbose=False):
    result = []
    split_text = input_text.split("\n")
    for t in split_text:
        if not t:
            continue
        res = process_sentence(t, rdrsegmenter, verbose=verbose)
        result.append(res["concepts"])
    return result

In [4]:
if "rdrsegmenter" not in globals():
    import py_vncorenlp
    rdrsegmenter = py_vncorenlp.VnCoreNLP(
        annotators=["wseg", "pos","ner","parse"],
        save_dir=r"E:\Github\uit_chatbot\graph\VnCoreNLP-1.2"
    )

In [14]:
dep_labels = set()
for i in range(10000):
    rows = extract_random_from_db(cursor, True)
    law = ""
    title = ""
    for r in rows:
        title += r['title'] + " "
        if not (r['title'].strip().startswith("Chương") or r['title'].strip().startswith("Điều")):
            if r['content']:
                law += r['content'] + "\n"

    text = clean_text(law)
    output = rdrsegmenter.annotate_text(text)

    for sentence in output.values():
        for token in sentence:
            dep_labels.add(token["depLabel"])

# in ra các nhãn duy nhất
for label in dep_labels:
    print(label)

mnr
iob
punct
loc
root
cnc
vmod
coord
xmdp
dir
dep
pob
conj
cnd
xtmp
adv
ext
prp
pmod
xadv
dob
amod
det
sub
xloc
tmp
x
nmod
prd
tpc


In [2]:
from neo4j import GraphDatabase

uri = "neo4j://127.0.0.1:7687"
username = "neo4j"
password = "1234567890"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [14]:
result = extract_triplet(rewrite_sentence, rdrsegmenter, True)
for r in result:
    print(r)

1. Original text:
 Giấy phép lái xe bao gồm các hạng sau đây. 

2. Cleaned text:
 giấy phép lái xe bao gồm các hạng sau đây. 

3. Segmented text (tokens):
 ['giấy_phép lái_xe bao_gồm các hạng sau đây .'] 

4. POS & Head & DepRel annotation:
Idx   Token           Head  DepRel          POS                      
---------------------------------------------------------------------------
1     giấy_phép       3     sub             Noun (Danh từ)           
2     lái_xe          1     nmod            Verb (Động từ)           
3     bao_gồm         0     root            Verb (Động từ)           
4     các             5     det             Determiner / article (Từ hạn định)
5     hạng            3     dob             Noun (Danh từ)           
6     sau             5     nmod            Noun (Danh từ)           
7     đây             5     det             Pronoun (Đại từ)         
8     .               3     punct           Punctuation (Dấu câu)    

5. Extracted triplets:
('giấy phép', 'lái x

In [ ]:
extract_triplet_and_store(rewrite_sentence, rdrsegmenter, driver, "ontology", id, so_hieu)

In [139]:
with driver.session(database="ontology") as session:
    session.execute_write(delete_all)